In [3]:
from MultidimentionalOpinionDynamics import Dynamics

import numpy as np

AgentsOpinions = np.matrix([[-1, 0, 8],
                        [1, 0, 10],
                        [1, 2, 20],
                        [20, -1, 0.56],
                        [-1, 0, -10]], dtype=np.float64)

GramMatrix = np.matrix([[1, 0, 0],
                        [0, 1, 0],
                        [0, 0, 1]], dtype=np.float64)
 


dynamics = Dynamics(AgentsOpinions, GramMatrix, 200)
dynamics.dist(AgentsOpinions, AgentsOpinions, GramMatrix)

Dynamics.dist(3, AgentsOpinions, AgentsOpinions, GramMatrix)

matrix([[  0.    ,   8.    , 152.    , 497.3536, 324.    ],
        [  8.    ,   0.    , 104.    , 451.1136, 404.    ],
        [152.    , 104.    ,   0.    , 747.9136, 908.    ],
        [497.3536, 451.1136, 747.9136,   0.    , 553.5136],
        [324.    , 404.    , 908.    , 553.5136,   0.    ]])